### Notebook Description: The Updated Approach

This notebook shifts from MediaPipe's default joint-based landmarks to midpoint sampling for RGB extraction in hand images, addressing initial accuracy limitations (<80%) by focusing on interphalangeal areas (e.g., mid-phalanges) to better capture skin color variations indicative of health status. It processes normal (`Hands_nl`) and abnormal (`Hands_abnl`) folders, assigning labels (1/0), calculating midpoints for thumb/index/middle/ring/pinky segments, and averaging RGB within radii (1, 3, 5 pixels) to generate clean CSVs (`hand_color_data_midpoints_radius_{radius}.csv`). Visualizations (e.g., green circles at midpoints) aid interpretability, with outputs saved to marked folders. Error handling for empty patches (NaNs) ensures robustness, though rare (e.g., thumb segments in specific images). 

T-tests confirm strong separation (p < 0.001 across channels/radii, green highest t ~5.9-6.0), validating the approach—healthy hands brighter (e.g., avg green gaps ~30 points). 

Overall, a solid preprocessing pipeline; radius 5 pixels likely optimal for smoothing noise while retaining signal, setting up downstream ML (e.g., RF achieving 92% accuracy). Adding HSV sampling in the future for complementary features may enhance predictive accuracy.


#### Methods

##### Image Acquisition and Labeling
Hand images were sourced from two directories: `Hands_nl` (normal/healthy hands, labeled as 1) and `Hands_abnl` (abnormal/unhealthy hands, labeled as 0), containing photographs in PNG, JPG, or JPEG formats. This binary classification reflects the project's ~127 valid images in the two categories were processed, ensuring case-insensitive extension handling for robustness.

##### Hand Landmark Detection
MediaPipe's Hands module (v0.10+, static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5) was employed to detect hand landmarks in RGB-converted images (cv2.cvtColor BGR to RGB). This open-source framework identifies 21 landmarks per hand, forming the basis for subsequent midpoint calculations.

##### Midpoint Calculation and RGB Sampling
To avoid joint areas (less representative of overall skin tone), midpoints were computed between consecutive landmarks for each finger segment:
- Thumb: Tip-IP (landmarks 4-3), IP-MCP (3-2).
- Index/Middle/Ring/Pinky: Tip-DIP (8-7/12-11/16-15/20-19), DIP-PIP (7-6/11-10/15-14/19-18), PIP-MCP (6-5/10-9/14-13/18-17).

Midpoint coordinates (cx, cy) were derived as the integer average of landmark pairs. RGB values were averaged over circular patches centered at each midpoint, with radii of 1, 3, and 5 pixels to evaluate smoothing effects (larger radii reduce noise from edges/shadows). Patch bounds were clamped to image dimensions (0 to height/width) to prevent out-of-bounds errors, with empty patches assigned NaN (imputed later via column means). This yielded ~84 features per image (RGB × segments × hands), stored in CSVs (`hand_color_data_midpoints_radius_{radius}.csv`).

##### Image Visualization
For qualitative assessment, original images were overlaid with MediaPipe's hand connections (mp_drawing.draw_landmarks) and green circles (cv2.circle, radius=5) at midpoints. Marked images were saved to `Hands_{nl/abnl}_marked_mid` folders, preserving filenames.

##### Statistical Analysis
Datasets were imputed (NaNs replaced with column means) and aggregated into average RGB channels (Avg_R/G/B). Independent t-tests (scipy.stats.ttest_ind) compared healthy vs. unhealthy groups per channel and radius, assessing significance (p < 0.05 threshold). Results were tabulated for inclusion in the report, highlighting channel-specific differences.

This method ensures focused, noise-reduced RGB sampling, enabling downstream binary classification while maintaining traceability to source images.

In [2]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import os

## Apply markings to photos. Dataset with points radius 3 pixels

In [3]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

# Function to extract averaged RGB around landmarks
def extract_averaged_rgb(image_path, radius=5):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to load image: {image_path}")
        return {}
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)
    if results.multi_hand_landmarks:
        landmarks = results.multi_hand_landmarks[0]
        h, w, _ = img_rgb.shape
        rgb_values = {}
        for i, lm in enumerate(landmarks.landmark):
            cx, cy = int(lm.x * w), int(lm.y * h)
            # Average RGB in radius, handling image boundaries
            x_start, x_end = max(0, cx - radius), min(w, cx + radius + 1)
            y_start, y_end = max(0, cy - radius), min(h, cy + radius + 1)
            patch = img_rgb[y_start:y_end, x_start:x_end]
            avg_rgb = np.mean(patch, axis=(0, 1)).astype(int)
            rgb_values[f'Landmark_{i}_R'] = avg_rgb[0]
            rgb_values[f'Landmark_{i}_G'] = avg_rgb[1]
            rgb_values[f'Landmark_{i}_B'] = avg_rgb[2]
        return rgb_values
    return {}



In [4]:
# Directory with photos 
photo_dir = '../Hands_nl'
data = []
valid_extensions = ('.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG')  # Case-insensitive extensions

for filename in os.listdir(photo_dir):
    if any(filename.lower().endswith(ext.lower()) for ext in valid_extensions):
        image_path = os.path.join(photo_dir, filename)
        rgb_values = extract_averaged_rgb(image_path, radius=5)  # Adjust radius
        if rgb_values:
            rgb_values['Source'] = filename
            rgb_values['Label'] = 0 if 'abnl' in filename.lower() else 1  # Case-insensitive label assignment
            data.append(rgb_values)

df_new = pd.DataFrame(data)
df_new.to_csv('../data/hand_color_data_expanded_radius5.csv', index=False)
print("Expanded data saved to 'hand_color_data_expanded_radius5.csv'")

Expanded data saved to 'hand_color_data_expanded_radius5.csv'


# Visualizing Photos with Points, Areas, and Size
MediaPipe can overlay landmarks on images. To visualize:

Points: Draw circles at landmarks.
Areas: Draw larger circles (e.g., radius=5) to show sampling areas.
Size Evaluation: Adjust circle size to represent sampling radius.

### Process normal hands photos

In [5]:
import cv2
import mediapipe as mp
import os

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

# Directory with photos and output (replace with folder paths)
photo_dir = '../Hands_nl'
output_dir = '../Hands_nl_marked'
os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist
valid_extensions = ('.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG')

for filename in os.listdir(photo_dir):
    if any(filename.lower().endswith(ext.lower()) for ext in valid_extensions):
        image_path = os.path.join(photo_dir, filename)
        img = cv2.imread(image_path)
        if img is None:
            print(f"Failed to load image: {image_path}")
            continue
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(img, landmarks, mp_hands.HAND_CONNECTIONS)
                # Draw larger areas (sampling radius)
                h, w, _ = img.shape
                for lm in landmarks.landmark:
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    cv2.circle(img, (cx, cy), 5, (0, 255, 0), 1)  # Green circle, adjust radius
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))  # Convert back to BGR for saving
print("Visualized photos saved to output folder.")

Visualized photos saved to output folder.


#### The photo below shows a photo with Mediapipes landmarks used initially for sampling. Some points land on interphalyngeal joints and have an unrepresentative hand color.

![](../images_and_graphs/hand_marked2.png)

### Adjust code to mark at midpoints interphalangeal joint and create folders of the images marked at sampling midpoints

Explanation of Changes

Multiple Folders: Added photo_dirs and output_dirs dictionaries to process both Hands_abnl and Hands_nl. The base path (base_photo_dir) adjustable to root.

Midpoint Marking: Replaced the joint landmark circles with midpoint calculations using the midpoint function. For each finger, it draws green circles (radius=5 pixels) at the midpoints between consecutive landmarks (e.g., Tip-DIP, DIP-PIP, PIP-MCP for index/middle/ring/pinky; Tip-IP, IP-MCP for thumb).

Joint Connections: Retained mp_drawing.draw_landmarks to show skeleton for context, with the focus on midpoint circles.

Output: Saves marked photos to Hands_abnl_marked_mid and Hands_nl_marked_mid, preserving the original filenames.

### Adjust the code to sample from the midpoints between the existing MediaPipe hand landmarks (interphalangeal areas) instead of the joint points themselves. This involves calculating the midpoint coordinates for each finger segment (distal, middle, proximal phalanges) and averaging RGB values around those midpoints.

Code adjustments:

Process both Hands_nl and Hands_abnl folders.

Assign Label based on folder name (0 for abnl, 1 for nl).

Include the Source column with the original filename.


Calculate midpoints for each finger segment:

Thumb: Between Tip-IP, IP-MCP.
Index/Middle/Ring/Pinky: Between Tip-DIP, DIP-PIP, PIP-MCP.


Sample averaged RGB at these midpoints with radii 1, 3, and 5.

Generate separate CSVs for radii 1, 3, and 5, sampling at midpoints between MediaPipe landmarks.

Handle multiple hands if detected.

In [6]:
import cv2
import mediapipe as mp
import os

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

# Directories with photos and output (replace with base path)
base_photo_dir = '../'  # Adjust this to base directory
photo_dirs = {'Hands_abnl': os.path.join(base_photo_dir, 'Hands_abnl_marked_mid'),'Hands_nl': os.path.join(base_photo_dir, 'Hands_nl_marked_mid')}

output_dirs = {'Hands_abnl': os.path.join(base_photo_dir, 'Hands_abnl_marked_mid'),'Hands_nl': os.path.join(base_photo_dir, 'Hands_nl_marked_mid')}

# Create output directories if they don't exist
for output_dir in output_dirs.values():
    os.makedirs(output_dir, exist_ok=True)

valid_extensions = ('.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG')

In [7]:
# Function to calculate midpoint between two landmarks
def midpoint(lm1, lm2, w, h):
    x1, y1 = int(lm1.x * w), int(lm1.y * h)
    x2, y2 = int(lm2.x * w), int(lm2.y * h)
    return (x1 + x2) // 2, (y1 + y2) // 2

for folder, photo_dir in photo_dirs.items():
    output_dir = output_dirs[folder]
    for filename in os.listdir(photo_dir):
        if any(filename.lower().endswith(ext.lower()) for ext in valid_extensions):
            image_path = os.path.join(photo_dir, filename)
            img = cv2.imread(image_path)
            if img is None:
                print(f"Failed to load image: {image_path}")
                continue
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            results = hands.process(img_rgb)
            if results.multi_hand_landmarks:
                for landmarks in results.multi_hand_landmarks:
                    h, w, _ = img.shape
                    fingers = {
                        'thumb': [4, 3, 2],  # Tip, IP, MCP
                        'index': [8, 7, 6, 5],  # Tip, DIP, PIP, MCP
                        'middle': [12, 11, 10, 9],
                        'ring': [16, 15, 14, 13],
                        'pinky': [20, 19, 18, 17]
                    }
                    # Draw midpoints
                    for lm_ids in fingers.values():
                        for segment in range(len(lm_ids) - 1):
                            lm1 = landmarks.landmark[lm_ids[segment]]
                            lm2 = landmarks.landmark[lm_ids[segment + 1]]
                            cx, cy = midpoint(lm1, lm2, w, h)
                            cv2.circle(img, (cx, cy), 5, (0, 255, 0), 1)  # Green circle at midpoint
                    # Optionally draw joint connections for context
                    mp_drawing.draw_landmarks(img, landmarks, mp_hands.HAND_CONNECTIONS)
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))  # Convert back to BGR
print("Visualized photos with midpoints saved to respective output folders.")

Visualized photos with midpoints saved to respective output folders.


#### Green circles added to photos to indicate sampling region to create dataset with radii 1, 3, and 5 pixels.

![](../images_and_graphs/hand_marked_mid.png)

# Obtain Datasets

In [8]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import os

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)

In [9]:
# Function to calculate midpoint between two landmarks
def midpoint(lm1, lm2, w, h):
    x1, y1 = int(lm1.x * w), int(lm1.y * h)
    x2, y2 = int(lm2.x * w), int(lm2.y * h)
    return (x1 + x2) // 2, (y1 + y2) // 2

In [11]:
# Function to extract averaged RGB at midpoints with error handling
def extract_averaged_rgb_at_midpoints(image_path, radius=3):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to load image: {image_path}")
        return {}
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w, _ = img_rgb.shape
    results = hands.process(img_rgb)
    if results.multi_hand_landmarks:
        rgb_values = {}
        for hand_idx, landmarks in enumerate(results.multi_hand_landmarks):
            # Define finger segments and their landmark IDs
            fingers = {
                'thumb': [4, 3, 2],  # Tip, IP, MCP
                'index': [8, 7, 6, 5],  # Tip, DIP, PIP, MCP
                'middle': [12, 11, 10, 9],
                'ring': [16, 15, 14, 13],
                'pinky': [20, 19, 18, 17]
            }
            for finger, lm_ids in fingers.items():
                for segment in range(len(lm_ids) - 1):
                    lm1 = landmarks.landmark[lm_ids[segment]]
                    lm2 = landmarks.landmark[lm_ids[segment + 1]]
                    cx, cy = midpoint(lm1, lm2, w, h)
                    # Define patch bounds with safety checks
                    x_start = max(0, cx - radius)
                    x_end = min(w, cx + radius + 1)
                    y_start = max(0, cy - radius)
                    y_end = min(h, cy + radius + 1)
                    patch = img_rgb[y_start:y_end, x_start:x_end]
                    if patch.size == 0:  # Check for empty patch
                        print(f"Empty patch at {finger}_segment_{segment}_{hand_idx} in {image_path}, using NaN")
                        avg_rgb = np.array([np.nan, np.nan, np.nan])
                    else:
                        avg_rgb = np.mean(patch, axis=(0, 1))
                    # No astype(int) - keep as float for NaN support
                    rgb_values[f'{finger}_segment_{segment}_{hand_idx}_R'] = avg_rgb[0]
                    rgb_values[f'{finger}_segment_{segment}_{hand_idx}_G'] = avg_rgb[1]
                    rgb_values[f'{finger}_segment_{segment}_{hand_idx}_B'] = avg_rgb[2]
        return rgb_values
    return {}

#### Process datasets with sampling radii 1,3 and 5.

In [13]:
# Directory with photos (replace with folder paths if changed)
photo_dirs = {
    'Hands_nl': 1,  # Label 1 for normal/Healthy
    'Hands_abnl': 0  # Label 0 for abnormal/Unhealthy
}
valid_extensions = ('.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG')

In [14]:
# Function to process datasets from both folders for different radii
def process_dataset(radius):
    data = []
    for folder, label in photo_dirs.items():
        folder_path = os.path.join('../', folder)  # Adjust base path
        for filename in os.listdir(folder_path):
            if any(filename.lower().endswith(ext.lower()) for ext in valid_extensions):
                image_path = os.path.join(folder_path, filename)
                rgb_values = extract_averaged_rgb_at_midpoints(image_path, radius=radius)
                if rgb_values:
                    rgb_values['Source'] = filename
                    rgb_values['Label'] = label
                    data.append(rgb_values)
    df_new = pd.DataFrame(data)
    output_file = f'../data/hand_color_data_midpoints_radius_{radius}.csv'
    df_new.to_csv(output_file, index=False)
    print(f"Dataset for radius {radius} saved to '{output_file}'")

### Make datasets for radii 1, 3, and 5

In [15]:
# Make datasets for radii 1, 3, and 5 pixels
for radius in [1, 3, 5]:
    process_dataset(radius)

Empty patch at thumb_segment_0_1 in ../Hands_abnl\hands_abnl_10.png, using NaN
Empty patch at thumb_segment_0_0 in ../Hands_abnl\IMG_9187.JPEG, using NaN
Dataset for radius 1 saved to '../data/hand_color_data_midpoints_radius_1.csv'
Empty patch at thumb_segment_0_1 in ../Hands_abnl\hands_abnl_10.png, using NaN
Empty patch at thumb_segment_0_0 in ../Hands_abnl\IMG_9187.JPEG, using NaN
Dataset for radius 3 saved to '../data/hand_color_data_midpoints_radius_3.csv'
Empty patch at thumb_segment_0_1 in ../Hands_abnl\hands_abnl_10.png, using NaN
Empty patch at thumb_segment_0_0 in ../Hands_abnl\IMG_9187.JPEG, using NaN
Dataset for radius 5 saved to '../data/hand_color_data_midpoints_radius_5.csv'


### T-test statistics

In [16]:
import pandas as pd
from scipy.stats import ttest_ind

for radius in [1, 3, 5]:
    df = pd.read_csv(f'../data/hand_color_data_midpoints_radius_{radius}.csv')
    df.fillna(df.mean(numeric_only=True), inplace=True)
    df['Avg_R'] = df.filter(regex='_R$').mean(axis=1)
    df['Avg_G'] = df.filter(regex='_G$').mean(axis=1)
    df['Avg_B'] = df.filter(regex='_B$').mean(axis=1)
    healthy = df[df['Label'] == 1]
    unhealthy = df[df['Label'] == 0]
    for channel in ['Avg_R', 'Avg_G', 'Avg_B']:
        t_stat, p_val = ttest_ind(healthy[channel], unhealthy[channel])
        print(f"Radius {radius}, {channel}: t={t_stat:.2f}, p={p_val:.4e}")

Radius 1, Avg_R: t=4.22, p=4.7240e-05
Radius 1, Avg_G: t=5.90, p=3.1813e-08
Radius 1, Avg_B: t=4.99, p=1.9682e-06
Radius 3, Avg_R: t=4.15, p=6.1038e-05
Radius 3, Avg_G: t=5.89, p=3.3506e-08
Radius 3, Avg_B: t=4.96, p=2.2910e-06
Radius 5, Avg_R: t=4.11, p=7.1264e-05
Radius 5, Avg_G: t=5.99, p=2.0721e-08
Radius 5, Avg_B: t=5.06, p=1.4491e-06


#### All p-values are highly significant (p < 0.001), confirming substantial differences in average RGB values between healthy and unhealthy hands across radii and channels. The green channel consistently shows the highest t-statistics (~5.9), indicating the strongest separation.

In [17]:
import pandas as pd
def df_to_markdown(df):
    # Header
    markdown = '| ' + ' | '.join(df.columns) + ' |\n'
    # Separator
    markdown += '| ' + ' | '.join(['---'] * len(df.columns)) + ' |\n'
    # Rows
    for _, row in df.iterrows():
        markdown += '| ' + ' | '.join(f'{val}' for val in row) + ' |\n'
    return markdown

data = {
    'Radius': [1, 1, 1, 3, 3, 3, 5, 5, 5],
    'Channel': ['Avg_R', 'Avg_G', 'Avg_B', 'Avg_R', 'Avg_G', 'Avg_B', 'Avg_R', 'Avg_G', 'Avg_B'],
    't-statistic': [4.22, 5.90, 4.99, 4.15, 5.89, 4.96, 4.11, 5.99, 5.06],
    'p-value': ['4.7240e-05', '3.1813e-08', '1.9682e-06', '6.1038e-05', '3.3506e-08', '2.2910e-06', '7.1264e-05', '2.0721e-08', '1.4491e-06']
}

df_stats = pd.DataFrame(data)

df_stats.to_csv('../data/stats', index=False)
print(df_to_markdown(df_stats))

| Radius | Channel | t-statistic | p-value |
| --- | --- | --- | --- |
| 1 | Avg_R | 4.22 | 4.7240e-05 |
| 1 | Avg_G | 5.9 | 3.1813e-08 |
| 1 | Avg_B | 4.99 | 1.9682e-06 |
| 3 | Avg_R | 4.15 | 6.1038e-05 |
| 3 | Avg_G | 5.89 | 3.3506e-08 |
| 3 | Avg_B | 4.96 | 2.2910e-06 |
| 5 | Avg_R | 4.11 | 7.1264e-05 |
| 5 | Avg_G | 5.99 | 2.0721e-08 |
| 5 | Avg_B | 5.06 | 1.4491e-06 |




### End of obtaining datasets